# core

> Main(/all) routines for `ipyfernel` 

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from jupyter_client.manager import KernelManager
from jupyter_client.kernelspec import KernelSpecManager
import subprocess 
from IPython.display import display, Image
import base64
from pathlib import Path
from IPython.core.magic import register_line_magic, register_line_cell_magic

## Remote Kernel Backend

In [ ]:
#| export
def set_ssh_config(
    port:int,                       # Port number on proxy server (e.g. bore.pub)
    user:str="",                    # Username on remote system.
    alias="remote_server_sshpyk",   # Default alias for `sshpyk`, leave it alone
    proxyname="bore.pub",           # Have tested this with bore
    config_path="~/.ssh/config",    # Shouldn't need to change this.
    ):
    config_path = Path(config_path).expanduser()
    if not config_path.exists(): config_path.touch()
    text = config_path.read_text()
    if f"Host {alias}" not in text: 
        assert user != "", "Must specify username when creating ~/.ssh/config info"
        block = f"""
Host {alias}
    HostName {proxyname}
    Port {port}
    User {user}
    BatchMode yes
    ControlMaster auto
    ControlPath ~/.ssh/sshpyk_%r@%h_%p
    ControlPersist 10m
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
"""
        config_path.write_text(text + block)
    else:
        lines = text.splitlines()
        in_target_block = False
        for i, line in enumerate(lines):
            if line.startswith("Host "):
                in_target_block = (line == f"Host {alias}")
            elif in_target_block and line.strip().startswith("Port "):
                lines[i] = f"    Port {port}"
            elif proxyname and in_target_block and line.strip().startswith("HostName "):
                lines[i] = f"    HostName {proxyname}"
            elif user and in_target_block and line.strip().startswith("User "):
                lines[i] = f"    User {user}"
        config_path.write_text("\n".join(lines) + "\n")
    print(f'{config_path} file updated.') 

In [ ]:
# Demo that:
port = 22756
set_ssh_config(port) 

/app/data/.ssh/config file updated.


In [ ]:
#| export
def register_remote_kernel(
    kernel_name="ipyf_remote_kernel",  # Any old name will do. This is fine.
    display_name="Remote Python",      # This is just what you'll see when you look at a list.
    remote_python="/path/to/python",  # Full path of Python executable to run on remote system.
    ssh_host_alias="remote_server_sshpyk", # Same alias as was used in writing to ssh config file.
    remote_kernel_name="python3",      # Typical for Jupiter.
    language="python",                 # Probably want to leave this unless you want to try R.
    verbose=True                       # Print extra info.
    ):
    ksm = KernelSpecManager()
    registered_names = list(ksm.get_all_specs().keys())
    if kernel_name in registered_names: 
        if verbose: print(f"{kernel_name} is already a registered kernel") 
    else: 
        if verbose: print(f"{kernel_name} is not a registered kernel. We need to add it") 
        subprocess.run(["sshpyk", "add", "--kernel-name", kernel_name,
            "--display-name", display_name, "--remote-python", remote_python, "--ssh-host-alias", ssh_host_alias,
            "--remote-kernel-name", remote_kernel_name, "--language", language
        ])
        if verbose: print("Success.")


In [ ]:
register_remote_kernel(remote_python='/Users/shawley/exercises/solveit/.venv/bin/python')

ipyf_remote_kernel is already a registered kernel


In [ ]:
#| export
_ipf_km, _ipf_kc = None, None            # "ipf" = "ipyfernel" ;-) 
def ipf_startup(kernel_name="ipyf_remote_kernel"):  
    "Start up the remote kernel"
    global _ipf_km, _ipf_kc 
    if _ipf_km is None and _ipf_kc is None: #only do this at startup
        _ipf_km = KernelManager(kernel_name=kernel_name)
        _ipf_km.start_kernel()
        _ipf_kc = _ipf_km.client()
        _ipf_kc.start_channels()
        _ipf_kc.wait_for_ready(timeout=30)
        print("Success: remote kernel started")
    else: 
        print("ipf_startup: already running")

In [ ]:
ipf_startup()

Success: remote kernel started


In [ ]:
#| export
def _output_hook(
    msg,   #  Message obtained from remote execution
    ):
    "How to handle output from the remote kernel."
    mt = msg["msg_type"]
    content = msg.get("content", {})
    if mt == "stream":
        print(content["text"], end="", flush=True)
    elif mt == "error":
        print('\n'.join(content.get("traceback", [])))
    elif mt in ("display_data", "update_display_data"):
        data = content.get("data", {})
        if "image/png" in data:
            display(Image(base64.b64decode(data["image/png"])))
        elif "text/plain" in data:
            print(data["text/plain"])

In [ ]:
#| export
def ipf_exec(
    code:str,           # Code to be executed
    verbose=False,      # Return details about remote execution.
    ):
    "Execute code on the remote kernel." 
    assert _ipf_kc is not None, "ipf_exec: need to run ipf_startup() first"
    result = _ipf_kc.execute_interactive(code=code, output_hook=_output_hook)
    _ipf_kc.last_result = result  # stash it for optional inspection later
    if verbose: return result

In [ ]:
code = """
import platform 
print(platform.system())
"""
ipf_exec(code)

Darwin


In [ ]:
#| export
def ipf_shutdown(verbose=True):
    "Terminates the remote kernel"
    global _ipf_km, _ipf_kc
    if verbose: print("Shutting down remote kernel") # Note: Could make say if remote kernel is not even running.
    try:
        if _ipf_kc is not None: _ipf_kc.stop_channels()
        if _ipf_km is not None: _ipf_km.shutdown_kernel(now=True)  # 'now=True' forces immediate shutdown
    except: pass  # Don't hang on errors
    _ipf_km, _ipf_kc = None, None

In [ ]:
ipf_shutdown()

Shutting down remote kernel


In [ ]:
#| export
def set_remote(port, user=""):
    """Setup connection to remote server, start remote server, and enable 'sticky' remote execution of code cells (even without magics)."""
    set_ssh_config(port, user=user) 
    try: 
        ipf_startup()
    except Exception as e: 
        print(f"Error starting up remote kernel: {e}") 
        return 

In [ ]:
set_remote(port)

/app/data/.ssh/config file updated.


Success: remote kernel started


# iPython Magics

In [ ]:
#| export
_skip_next = False  # This is used in conjunction with %%local, below

def _execute_remotely(lines:list[str]):
    "Take commands from magics and send to ipf_exec"
    global _skip_next
    if _skip_next:
        _skip_next = False
        return lines
    code = ''.join(lines)
    if 'get_ipython()' in code: return lines  # let solveit internals pass through
    # Make sure our magics execute locally
    if code.strip().startswith(('%local', '%%local', 'unset_remote(', 'set_remote(', 'set_sticky(','unset_sticky(')):
        return lines
    return [f"ipf_exec({repr(code)})\n"]

In [ ]:
#| export
@register_line_cell_magic
def remote(line, cell=None):
    "remote exeuction: works as %remote and as %%remote" 
    ipf_exec(cell if cell else line)

In [ ]:
%%remote 
#%%remote    <-- docs are filtering magics but that's what's used here
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

Hello from Chonk


In [ ]:
#| export
@register_line_cell_magic
def local(line, cell=None):
    "local execution: works as %local and as %%local"
    global _skip_next
    _skip_next = True
    get_ipython().run_cell(cell if cell else line) 

In [ ]:
%%local 
#%%local <-- docs are filtering magics but that's what's used here
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

Hello from 909e93a3a71e


In [ ]:
#| export
def unset_remote():
    "shutdown remote server"
    unset_sticky()  # get rid of any input transformers (see below) 
    ipf_shutdown()

## 'Sticky'/'Seamless' Remote Excution 

via Input Transformers.  These can make cells set execute remotely by default.

**WARNINGS**: 
1. Solve it is not intended to work with people modifying input transformers So be wary. Nevertheless, this seems to work.
2. If they're commands that you definitely want to execute locally, maybe run `%unset_sticky` just to be sure first.


In [ ]:
#| export
gip = get_ipython()

def set_sticky():
    "Makes code cells execute remotely, via input transformer"
    assert _ipf_kc is not None, "Need an active remote kernel connection" 
    for f in gip.input_transformers_cleanup[:]:   # gaurd against appending twice
        if getattr(f, '__name__', '') == '_execute_remotely':
            print("Already executing remotely") 
            return 
    gip.input_transformers_cleanup.append(_execute_remotely)
    print('Code cells will now execute remotely.')

In [ ]:
#| export
def unset_sticky():
    "Un-sticks remote execution for code cells" 
    for f in gip.input_transformers_cleanup[:]:  
        if getattr(f, '__name__', '') == '_execute_remotely':
            gip.input_transformers_cleanup.remove(f)
    print("Code cells will now run locally.") 



In [ ]:
unset_remote()

print()
set_remote(port)
set_sticky()


Code cells will now run locally.
Shutting down remote kernel



/app/data/.ssh/config file updated.


Success: remote kernel started
Code cells will now execute remotely.


In [ ]:
# remote execution
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from Chonk


In [ ]:
unset_sticky()

Code cells will now run locally.


In [ ]:
# local execution
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

In [ ]:
#| hide

#import nbdev; nbdev.nbdev_export()
!nbdev_export --procs scrub_magics

/opt/homebrew/lib/python3.10/site-packages/nbdev/doclinks.py:17: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib


usage: nbdev_export [-h] [--path PATH] [--symlinks] [--file_glob FILE_GLOB]
                    [--file_re FILE_RE] [--folder_re FOLDER_RE]
                    [--skip_file_glob SKIP_FILE_GLOB]
                    [--skip_file_re SKIP_FILE_RE]
                    [--skip_folder_re SKIP_FOLDER_RE]
nbdev_export: error: unrecognized arguments: --procs scrub_magics


You know, it occurs to me that apart from the cell magics, remote and local, none of the other magics we define really need to be magics, do they? They could just be ordinary functions, or am I wrong?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're absolutely right! Let's look at each one:

- **`set_remote`** - Just calls `set_ssh_config()` and `ipf_startup()`. Could easily be a plain function.
- **`unset_remote`** - Just calls `unset_sticky()` and `ipf_shutdown()`. Same deal.
- **`set_sticky`** / **`unset_sticky`** - These manipulate `get_ipython().input_transformers_cleanup`. Could be functions that call `get_ipython()` internally.

The only ones that genuinely benefit from being magics are `%%remote` and `%%local` — because they need to capture the *cell content* as their argument, which is what cell magics do naturally.

The line magic versions (`%remote` and `%local`) could technically be functions too, but the cell magic variants couldn't easily be replaced.

So yes — you could simplify by making `set_remote`, `unset_remote`, `set_sticky`, and `unset_sticky` into regular functions. The main thing you'd lose is the `%` prefix syntax, which some folks find convenient/readable, but that's purely aesthetic.

Yeah, the problem I'm having is that the NB Dev Documentation website builder filters out all magics so that none of them appear in the final documentation, and the only way around it, and believe me, I've spent half a day on this. Only way I can get this to happen is to just add them in again as comments to show what the magic would have looked like if it had been rendered in the documentation, which is kind of stupid. So if it were only remote and local, where I still needed to do that little comment documentation hack, then all the other routines would presumably just appear normally in the documentation.


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Ah, that's a really practical reason to make the change! If `set_remote`, `unset_remote`, `set_sticky`, and `unset_sticky` become regular functions, they'll render normally in the nbdev docs without any hacks.

You'd only need your comment workaround for `%%remote` and `%%local` — the two that genuinely *must* be cell magics.

Want to refactor those four into plain functions? The changes would be minimal — basically just removing the `@register_line_magic` decorators and changing the signature from `def set_sticky(_):` to `def set_sticky():` (dropping the unused line argument).

Yeah, the only one I have to be careful about is unset sticky. You do need to wrap it in a local magic or else it will try to execute on the remote system. Previously when we had it as a magic it was easier to filter.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good catch! When sticky mode is active, calling `unset_sticky()` as a plain function would get intercepted by `_execute_remotely` and sent to the remote kernel — which defeats the purpose entirely.

A few options:

1. **Keep requiring `%%local`** before `unset_sticky()` — simple, but users have to remember it
2. **Add `unset_sticky` to the filter in `_execute_remotely`** — you're already checking for magic names there, you could add `'unset_sticky('` to that condition
3. **Keep `unset_sticky` as a magic** — then the existing filter catches it automatically

Option 2 seems like a nice middle ground — minimal change, and users can just call `unset_sticky()` naturally. What do you think?

Hm, I tried trapping for it as you suggested in number two, but it doesn't seem to have taken.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Hmm, looking at your `_execute_remotely` function, it uses `code.strip().startswith(...)` — which means it only catches `unset_sticky(` if it's at the very *beginning* of the cell.

In your test cell you had:
```python
#%%local
unset_sticky()
```

The comment comes first, so `.startswith('unset_sticky(')` fails.

You could switch from `startswith` to checking if the string is anywhere in the code, using `in` instead — or would you prefer a different approach?